In [2]:
import numpy as np
import pandas as pd
import sklearn
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import regexp_tokenize
import re
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV


from sklearn.svm import SVC


nltk.download('stopwords')
stops = set(stopwords.words('spanish'))
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
df_todo = pd.read_csv('/content/drive/MyDrive/5to semestre/PNL 2/TP-2/data/report.csv', sep=';', encoding='utf-8')
df_SwL = pd.read_csv('/content/drive/MyDrive/5to semestre/PNL 2/TP-2/data/report.csv', sep=';', encoding='utf-8')

In [ ]:
df_todo

,RN,TWEET,CATEGORIA
0,5,o sea va a votar a javier milei,Positivo
1,6,presideeeente peluca presideeente peluca presi...,Positivo
2,8,es con el milei sin dudas,Positivo
3,10,lo e escuchado hablar de casta política alguie...,Negativo
4,11,es mi lamentable deber informar que javier mil...,Negativo
...,...,...,...
398,673,estafadores son vos y el facho de milei que co...,Negativo
399,674,beto no entiendo cómo un tipo inteligente como...,Negativo
400,675,milei es un payaso engaña bobos un facho y nos...,Negativo
401,676,ideas de un facho decimonónico cualquiera,Negativo


In [ ]:
df_todo =df_todo.drop(['RN'], axis= 1)
display(df_todo.shape)
df_SwL =df_SwL.drop(['RN'], axis= 1)
display(df_SwL.shape)

(403, 2)

(403, 2)

In [ ]:
df_todo.isnull().sum()

TWEET        0
CATEGORIA    0
dtype: int64

In [1]:
def filterSW(corpus):

    tokenizador_expreg = nltk.regexp_tokenize
    texto = corpus.lower()
    tokens = tokenizador_expreg(re.sub("\d+", " ", texto), pattern='\w+')
    tokens = [token.strip() for token in tokens]
    tokens = [token for token in tokens if token.lower() not in stopwords.words("spanish")]
    union = ' '.join(tokens)

    return union

In [3]:
def lemmatization(corpus):
    wnl = WordNetLemmatizer()
    oraciones = [wnl.lemmatize(oracion) for oracion in corpus ]
    #union = ' '.join(oraciones)

    return oraciones

In [ ]:
# Importamos el SnowballStemmer
def stemm(df):

    from nltk.stem import SnowballStemmer
    spanish_stemmer = SnowballStemmer('spanish')

    oraciones=[]

    for oracion in df['TWEET']:
        oracion_stemm = []
        for palabra in oracion.split():
            oracion_stemm.append(spanish_stemmer.stem(palabra))
        oraciones.append(" ".join(oracion_stemm))

    return oraciones

In [4]:
def preprocesadorTodo(df):
    df['TWEET'] = df['TWEET'].apply(filterSW)
    df['TWEET'] = lemmatization(df['TWEET'])
    df['TWEET'] = stemm(df)
    #df = df.drop_duplicates().reset_index(drop=True)

    return df['TWEET']

In [ ]:
def preprocesadorSwL(df):
    df['TWEET'] = df['TWEET'].apply(filterSW)
    df['TWEET'] = lemmatization(df['TWEET'])
    #df = df.drop_duplicates().reset_index(drop=True)

    return df['TWEET']

In [ ]:
df_todo['TWEET'] = preprocesadorTodo(df_todo)
df_SwL['TWEET'] = preprocesadorSwL(df_SwL)

In [5]:
def Generar_Matriz_BOW(df, binario=False, ngram=(1,1)):
    """
    Recibe un DataFrame con el corpus etiquetado a procesar (típicamente dos columnas:
    Oración y Categoria). El segundo parámetro indica el nombre de la columna a modelar, o sea,
    la columna que tiene los documentos a escanear para generar la matriz.
    Devuelve un objeto Vectorizador entrenado con la matriz BOW para las oraciones recibidas, y
    también el dataframe recibido al que se añaden tantas columnas como tokens se hayan identificado
    en el proceso de generación de la matriz, etiquetando cada una con el nombre de dicho token,
    y con sus cantidades contabilizadas en la matriz.
    El parámetro binario establece si cuenta cada ocurrencia del token en el documento, o si sólo
    cuenta uno o cero, sin importar cuántas veces esté presente la misma palabra en el documento.
    El parámetro ngram es una tupla (m,n) que establece el tamaño mínimo y máximo de los ngrams
    a considerar. Para tokenizar sólo con palabras individuales pasar (1,1), para sólo bigramas
    pasar (2,2), para sólo trigramas pasar (3,3), para incluir los tres anteriores pasar (1,3).
    """

    # Vectorizar, usando CountVectorizer de sklearn.feature_extraction.text
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizador = CountVectorizer(binary=binario, ngram_range=ngram)
    X = vectorizador.fit_transform(df['TWEET'])

    # Generar el DataFrame a devolver
    df_X = pd.DataFrame(X.toarray(), columns=vectorizador.get_feature_names())
    df_bow = df.join(df_X)

    return vectorizador, df_bow

In [6]:
df_todo_vectorizador, df_todo_bow = Generar_Matriz_BOW(df_todo)
df_SwL_vectorizador, df_SwL_bow = Generar_Matriz_BOW(df_SwL)

NameError: ignored

In [ ]:
def Generar_Matriz_Tfidf(df, ngram=(1,1)):
    """
    Recibe un DataFrame con el corpus etiquetado a procesar (típicamente dos columnas:
    Oración y Categoria). El segundo parámetro indica el nombre de la columna a modelar, o sea,
    la columna que tiene los documentos a escanear para generar la matriz.
    Devuelve un objeto Vectorizador entrenado con la matriz Tf*Idf para las oraciones recibidas, y
    también el dataframe recibido al que se añaden tantas columnas como tokens se hayan identificado
    en el proceso de generación de la matriz, etiquetando cada una con el nombre de dicho token,
    y con sus cantidades contabilizadas en la matriz.
    El parámetro ngram es una tupla (m,n) que establece el tamaño mínimo y máximo de los ngrams
    a considerar. Para tokenizar sólo con palabras individuales pasar (1,1), para sólo bigramas
    pasar (2,2), para sólo trigramas pasar (3,3), para incluir los tres anteriores pasar (1,3).
    """

    # Vectorizar... Directamente usar aquí el TfidfVectorizer de sklearn en vez del CountVectorizer
    # (Lleva los mismos parámetros y directamente nos devuelve la matriz con los vectores Tf*Idf)
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizador = TfidfVectorizer(ngram_range=ngram)
    X = vectorizador.fit_transform(df['TWEET'])

    # Generar el DataFrame a devolver
    df_X = pd.DataFrame(X.toarray(), columns=vectorizador.get_feature_names())
    df_Tfidf = df.join(df_X)

    return vectorizador, df_Tfidf

In [ ]:
df_todo_vectorizador, df_todo_Tfidf = Generar_Matriz_Tfidf(df_todo)
df_SwL_vectorizador, df_SwL_Tfidf = Generar_Matriz_Tfidf(df_SwL)

In [ ]:
df_todo_Tfidf = df_todo_Tfidf.drop("TWEET", axis = 1)
df_SwL_Tfidf = df_SwL_Tfidf.drop("TWEET", axis = 1)

In [ ]:
df_todo_Tfidf

,CATEGORIA,abaj,abascal,aberr,abog,abran,abraz,abrir,absolut,aca,...,xenofobi,xx,ye,yofr,zapat,zooland,zorr,zurd,ñoqui,ñoquis
0,Positivo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Positivo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Positivo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Negativo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Negativo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,Negativo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
399,Negativo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,Negativo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
401,Negativo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Separar el corpus en Train/Test
X_todo_Tfidf = df_todo_Tfidf.drop(['CATEGORIA'],axis=1)
y_todo_Tfidf = df_todo_Tfidf[['CATEGORIA']]
#y_todo_Tfidf = pd.get_dummies(y_todo_Tfidf)
X_todo_Tfidf_train, X_todo_Tfidf_test, y_todo_Tfidf_train, y_todo_Tfidf_test = train_test_split(X_todo_Tfidf, y_todo_Tfidf, test_size=0.20, random_state=124)

In [9]:
# Separar el corpus en Train/Test
X_SwL_Tfidf = df_SwL_Tfidf.drop(['CATEGORIA'],axis=1)
y_SwL_Tfidf = df_SwL_Tfidf[['CATEGORIA']]
#y_SwL_Tfidf = pd.get_dummies(y_todo_Tfidf)
X_SwL_Tfidf_train, X_SwL_Tfidf_test, y_SwL_Tfidf_train, y_SwL_Tfidf_test = train_test_split(X_SwL_Tfidf, y_SwL_Tfidf, test_size=0.20, random_state=124)

NameError: ignored

In [7]:
# kNN Todo
# Creamos el modelo con los parámetros que no cambiarán, observe que no le pasamos el valor de C de regulrización
knn= KNeighborsClassifier(n_jobs=-1)
# Armamos el diccionario con el  nombre y valores para los Hiperparámetros
parametros_kNN = {'n_neighbors':[2,3,4,5,6],
                   'p' : [1,2],
                  'weights': ['uniform', 'distance'],
                  'metric': ['minkowski','cityblock', 'euclidean']}
# Armamos el GridSearchCV
grid_kNN = GridSearchCV(estimator = knn, param_grid = parametros_kNN,scoring = 'accuracy', cv = 5, verbose = 1,
                        n_jobs = -1)
# Lo entrenamos en el Train Set (él lo dividirá internamente y en cada fold estarán en ValTrain y ValTest)
grid_kNN.fit(X_todo_Tfidf_train,y_todo_Tfidf_train);
# Acc
AC_kNN_best = grid_kNN.best_score_
#Parametros
parametros_kNN_best = grid_kNN.best_params_

NameError: ignored

In [ ]:
print(f"kNN: {parametros_kNN_best}")
print(f"kNN Acc: {AC_kNN_best}")

kNN: {'metric': 'minkowski', 'n_neighbors': 6, 'p': 2, 'weights': 'distance'}
kNN Acc: 0.7297596153846154


In [10]:
# kNN SwL
# Creamos el modelo con los parámetros que no cambiarán, observe que no le pasamos el valor de C de regulrización
knn= KNeighborsClassifier(n_jobs=-1)
# Armamos el diccionario con el  nombre y valores para los Hiperparámetros
parametros_kNN = {'n_neighbors':[2,3,4,5,6],
                   'p' : [1,2],
                  'weights': ['uniform', 'distance'],
                  'metric': ['minkowski','cityblock', 'euclidean']}
# Armamos el GridSearchCV
grid_kNN = GridSearchCV(estimator = knn, param_grid = parametros_kNN,scoring = 'accuracy', cv = 5, verbose = 1,
                        n_jobs = -1)
# Lo entrenamos en el Train Set (él lo dividirá internamente y en cada fold estarán en ValTrain y ValTest)
grid_kNN.fit(X_SwL_Tfidf_train, y_SwL_Tfidf_train);
# Acc
AC_kNN_best = grid_kNN.best_score_
#Parametros
parametros_kNN_best = grid_kNN.best_params_

NameError: ignored

In [11]:
print(f"kNN: {parametros_kNN_best}")
print(f"kNN Acc: {AC_kNN_best}")

NameError: ignored

In [8]:
# SVM todo

# Creamos el modelo con los parámetros que no cambiarán, observe que no le pasamos el valor de C de regulrización
svm = SVC( random_state=123)
# Armamos el diccionario con el  nombre y valores para los Hiperparámetros
parametros_svm = {'C' : [1e-2,1e-1,1,1e1,1e10]}
# Armamos el GridSearchCV
grid_svm = GridSearchCV(estimator = svm, param_grid = parametros_svm, scoring = 'accuracy', cv = 5, verbose = 1,
                        n_jobs = -1)

# Lo entrenamos en el Train Set (él lo dividirá internamente y en cada fold estarán en ValTrain y ValTest)
grid_svm.fit(X_todo_Tfidf_train, y_todo_Tfidf_train);
# Acc
AC_svm_best = grid_svm.best_score_

#Parametros
parametros_svm_best = grid_svm.best_params_


NameError: ignored

In [ ]:
print(f"SVC: {parametros_svm_best}")
print(f"SVC Acc: {AC_svm_best}")

SVC: {'C': 10.0}
SVC Acc: 0.7237019230769232


In [ ]:
# SVM SwL

# Creamos el modelo con los parámetros que no cambiarán, observe que no le pasamos el valor de C de regulrización
svm = SVC( random_state=123)
# Armamos el diccionario con el  nombre y valores para los Hiperparámetros
parametros_svm = {'C' : [1e-2,1e-1,1,1e1,1e10]}
# Armamos el GridSearchCV
grid_svm = GridSearchCV(estimator = svm, param_grid = parametros_svm, scoring = 'accuracy', cv = 5, verbose = 1,
                        n_jobs = -1)

# Lo entrenamos en el Train Set (él lo dividirá internamente y en cada fold estarán en ValTrain y ValTest)
grid_svm.fit(X_SwL_Tfidf_train, y_SwL_Tfidf_train)
# Acc
AC_svm_best = grid_svm.best_score_

#Parametros
parametros_svm_best = grid_svm.best_params_


Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print(f"SVC: {parametros_svm_best}")
print(f"SVC Acc: {AC_svm_best}")

SVC: {'C': 10.0}
SVC Acc: 0.720576923076923


# Modelo produccion.

In [ ]:
knnProdu= KNeighborsClassifier(n_jobs=-1, metric =  'minkowski', n_neighbors = 5, p = 2, weights =  'uniform')
knnProdu.fit(X_SwL_Tfidf, y_SwL_Tfidf)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_jobs=-1)

In [ ]:
nuevo = 'me gusta como pienza javier milei'

In [ ]:
dfNuevos = pd.DataFrame()
dfNuevos

""


In [ ]:
dfNuevos['TWEET'] = [nuevo]
dfNuevos['CatReal'] = ['POSITIVO']
dfNuevos

,TWEET,CatReal
0,me gusta como pienza javier milei,POSITIVO


In [ ]:
dfNuevos['TWEET'] = preprocesadorSwL(dfNuevos)
dfNuevos

,TWEET,CatReal
0,gusta pienza javier milei,POSITIVO


In [ ]:
df_todo_vectorizador, df_todo_Tfidf = Generar_Matriz_Tfidf(df_SwL)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
dfNuevos_vectorizador, dfNuevos_Tfidf = Generar_Matriz_Tfidf(dfNuevos)

In [ ]:
nuevodat =  df_SwL_vectorizador.transform(dfNuevos['TWEET'])
nuevodat = nuevodat.todense()

In [ ]:
X_dfNuevos_Tfidf = dfNuevos_Tfidf.drop(['CatReal','TWEET'],axis=1)

In [ ]:
 valor = knnProdu.predict(nuevodat)
 valor

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


array(['Positivo'], dtype=object)

In [ ]:
df_test = pd.read_excel('/content/drive/MyDrive/5to semestre/PNL 2/TP-2/data/twits_test.xlsx')

In [ ]:
df_test['TWEET'] = preprocesadorSwL(df_test)

In [ ]:
df_test_tfidf =  df_SwL_vectorizador.transform(df_test)
df_test_tfidf = df_test_tfidf.todense()

In [ ]:
for x in range(len(df_test['TWEET'])):
  tfidf = df_SwL_vectorizador.transform([df_test.TWEET.iloc[x]])
  valor = knnProdu.predict(tfidf)
  if valor[0] == 'Negativo':
    df_test.loc[x, 'CatModel']  = 'NEGATIVO'
  if valor[0] == 'Positivo':
    df_test.loc[x, 'CatModel']  = 'POSITIVO'

In [ ]:
matrixConfucion = df_test.groupby(['CatReal','CatModel'])['CatModel'].count()

CatReal   CatModel
NEGATIVO  NEGATIVO     3
          POSITIVO     9
POSITIVO  NEGATIVO     2
          POSITIVO    14
Name: CatModel, dtype: int64

In [ ]:
df_matrixConfucion = pd.DataFrame({'count' : df_test.groupby(['CatReal','CatModel'])['CatModel'].count()}).reset_index()
df_matrixConfucion

,CatReal,CatModel,count
0,NEGATIVO,NEGATIVO,3
1,NEGATIVO,POSITIVO,9
2,POSITIVO,NEGATIVO,2
3,POSITIVO,POSITIVO,14
